# Exploratory Data Analysis

In this notebook, we will explore the dataset provided and devise some simple features to use in the model.

## First look

### Size

This dataset easily fits entirely in memory on most devices.
There are 102 homes in the `homes` table.
There are 580k sensor observations in the `motion` table, for only 50 homes.

### Schema + Types

The `datetime` column was timezone-aware, I have normalized this to UTC to make it easier to work with in Pandas.

Our response is an integer indicating two potential outcomes:

* 0: there was a single occupant in the house
* 1: there was multiple occupancy

There does not seem to be an option for zero occupancy, although this is probably indicated by times when no sensors were triggered.

### Cleaning

Homes with no `motion` observation are not interesting for fitting the model as there are no possible features to use, so I dropped them from the analysis even though this reduces the available samples by more than half.

### Response Distribution

After removing homes without sensor data, we are left with a slightly larger number of multiple-occupancy homes.

In [1]:
import sqlite3
import pandas as pd
import numpy as np
from lib.data.features import read_raw_data
from lib.common.paths import DATABASE_LOCATION
from lib.data.raw import download_raw_data_if_not_exists
from lib.data.split import add_train_valid_test_split_table

download_raw_data_if_not_exists(DATABASE_LOCATION)
add_train_valid_test_split_table(DATABASE_LOCATION)

df = read_raw_data(DATABASE_LOCATION, train=True)

print(f"""
Raw data summary:
Shape: {df.shape}
Schema + Types: {df.dtypes.to_dict()}
Missing Data:{df.isnull().sum().to_dict()}
Duplicated primary key: {sum(df.duplicated(["id", "home_id"]))}
Duplicated home_id + location + time: {sum(df.duplicated(["home_id", "location", "datetime"]))}
Response mean: {df.groupby("home_id").agg({"multiple_occupancy": "first"})["multiple_occupancy"].mean()}
""")

2024-04-30 21:12:16,643 - INFO - Raw data downloaded successfully to /home/jovyan/work/data/data.db
2024-04-30 21:12:16,780 - INFO - Table train_valid_test added successfully.



Raw data summary:
Shape: (288016, 5)
Schema + Types: {'home_id': dtype('O'), 'multiple_occupancy': dtype('int64'), 'id': dtype('O'), 'datetime': datetime64[ns, UTC], 'location': dtype('O')}
Missing Data:{'home_id': 0, 'multiple_occupancy': 0, 'id': 0, 'datetime': 0, 'location': 0}
Duplicated primary key: 0
Duplicated home_id + location + time: 0
Response mean: 0.76



## Train-Valid-Test Split

It is critical to ensure that all steps taken when developing a model have not been tainted with information from the response variable.

A common practice is to leave a significant sample of data unseen until all decisions relating to the model have been taken and it is ready to be assessed. This will be the test set.

I also include a validation set, which will be used to benchmark multiple models and feature combinations to select the final model.

The dataset is split by `home_id`, as my prior expectation is that each `home_id` would operate independently of each other. 

This avoids issues where the model has been trained on data containing that particular `home_id` which would have artificially higher performance in unseen samples.

A good example of this is the case where a test `datetime` occurs between two training `datetime` in that same `home_id` and `location`.